In [1]:
pip install python-dotenv

In [2]:
# import os
# from dotenv import load_dotenv, find_dotenv
# _= load_dotenv(find_dotenv())

# ap =os.getenv("MY_VAR")


In [3]:
# !pip install langchain
# !pip install openai

In [4]:
!pip install -U langchain-openai
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 693.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succes

In [5]:
# prompt: 어떻게 api_key를 넣는가?
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

# Load the environment variables from the .env file
load_dotenv(find_dotenv())

# Get the OpenAI API key from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

In [7]:
url = "https://github.com/tlorans/ClimateRisks/blob/main/DEXUSEU.csv?raw=true"

import pandas as pd
data = pd.read_csv(url)
data.rename(columns = {"DEXUSEU":"Exchange Rate Dollar to Euro"}, inplace = True)
data.to_csv("exchange_rate.csv", index = None)

In [8]:
from langchain.document_loaders import CSVLoader

file = "exchange_rate.csv"
loader = CSVLoader(file_path = file)
data = loader.load()
print(data[0])

ModuleNotFoundError: Module langchain_community.document_loaders not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

Custom Tools

In [ ]:
!pip install docarray
!pip install tiktoken

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
).from_loaders([loader])


In [ ]:
index.query("What is the last exchange rate? Gives the date")

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0282881... -0.019532937125661218]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0282881... -0.019532937125661218]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

In [ ]:
pip install -U docarray==0.21.

  Using cached docarray-0.21.0-py3-none-any.whl
  Attempting uninstall: docarray
    Found existing installation: docarray 0.40.0
    Uninstalling docarray-0.40.0:
      Successfully uninstalled docarray-0.40.0


In [ ]:
from docarray import DocumentArray, Document

# query_text를 사용하여 Document 객체 생성
query_text = "What is the last exchange rate? Gives the date"
query_doc = Document(text=query_text)

# Document 객체를 DocumentArray에 추가
docs = DocumentArray([query_doc])

# DocumentArray를 사용하여 쿼리 실행
response = index.query(docs)

print(response)



ImportError: Cannot import name 'DocumentArray' from 'docarray'.
The object named 'DocumentArray' does not exist anymore in this version of docarray.
If you still want to use 'DocumentArray' please downgrade to version <=0.21.0 with: `pip install -U docarray==0.21.0`.

In [ ]:
from langchain.tools import Tool

knowledge_tool = Tool(
    name = "FX Rate",
    func = index.query,
    description = "Useful for when you need to search for the exchange rate. Provides your input as a search query."
)

In [ ]:
from langchain.agents import AgentType
from langchain.agents import load_tools, initialize_agent
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature = 0.)


tools = [knowledge_tool]

agent = initialize_agent(
    tools,
    chat,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

agent.run("What was the exchange rate in April 2022?")



> Entering new AgentExecutor chain...
Thought: We need to find the exchange rate for April 2022. Let's use the FX Rate tool to search for this information. 

Action:
```
{
  "action": "FX Rate",
  "action_input": "Exchange rate April 2022"
}
```



ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0111796... -0.006469333319814567]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0111796... -0.006469333319814567]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

Interacting with Other tools

In [ ]:
!pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 79.8 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.4
    Uninstalling lxml-4.9.4:
      Successfully uninstalled lxml-4.9.4


In [ ]:
tools = load_tools(["llm-math"], llm = chat)

from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

duckduckgo_tool = Tool(
    name = 'DuckDuckGo Search',
    func = search.run,
    description = "Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

tools.append(knowledge_tool)
tools.append(duckduckgo_tool)

agent = initialize_agent(
    tools,
    chat,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

In [ ]:
query = """Process as follow:\n\
1. Search for Tesla's revenue in 2022. \n\
2. Find the exchange rate in December 2022. \n\
3. Multiply Tesla's revenue in 2022 with the exchange rate. \n\
"""
agent.run(query)



> Entering new AgentExecutor chain...
Question: What is the process to calculate the value of Tesla's revenue in December 2022?
Thought: We need to find Tesla's revenue in 2022, then the exchange rate in December 2022, and finally multiply the two values.
Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "Tesla revenue in 2022"
}
```

Observation: Tesla's annual revenue has grown steadily since 2008, peaking in 2022 after breaking records in 2021. In 2018, Tesla's revenue stood at over 21 billion USD. Another major source of revenue for Tesla is the sale of regulatory credits to other automotive manufacturers, which brought in $1.8 billion in revenues in 2022. Tesla earns tradable credits under ... Automotive sales are Tesla's strongest growth driver, accounting for about 82.5 percent of revenue in 2022. Automotive sales revenue increased by around 52 percent year-on-year. No loss on vision Tesla ended Q4 2023 with a net income of $7.9 billion and the full year with $1

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0043584... -0.003923557649910808]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0043584... -0.003923557649910808]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

연습

In [ ]:
url = "	https://pasteur.epa.gov/uploads/10.23719/1528686/SupplyChainGHGEmissionFactors_v1.2_NAICS_CO2e_USD2021.csv"

import pandas as pd
emissions = pd.read_csv(url)

emissions.to_csv("emissions_factors.csv", index = None)

file = "emissions_factors.csv"
loader = CSVLoader(file_path = file)
data = loader.load()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
index.query("What is the emission factor with margins for automobile industry?")

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [0.00682050... -0.028722258929805827]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [0.00682050... -0.028722258929805827]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

In [ ]:
!pip install openai
!pip install langchain
!pip install docarray
!pip install tiktoken
!pip install duckduckgo-search

In [ ]:
from langchain.tools import Tool

knowledge_tool = Tool(
    name = "Supply Chain Emissions Factors",
    func = index.query,
    description = "Useful for when you need to find the emissions supply-chain for a specific industry. You want to return the emissions factors from the \
    most related industry."
)

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

duckduckgo_tool = Tool(
    name = 'DuckDuckGo Search',
    func = search.run,
    description = "Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

In [ ]:
from langchain.agents import AgentType
from langchain.agents import load_tools, initialize_agent
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature = 0.)
tools = load_tools(["llm-math"], llm = chat)

tools.append(knowledge_tool)
tools.append(duckduckgo_tool)

agent = initialize_agent(
    tools,
    chat,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose = True
)

In [ ]:
agent.run("""Please follow the following process: \n\
1. Find Tesla's main activity and last available revenue.\n\
2. Find the emissions factor with margin corresponding to Tesla's activity\n\
3. Multiply the emissions factor with the revenue\n\
Result:
""")



> Entering new AgentExecutor chain...
Question: Please follow the following process:
1. Find Tesla's main activity and last available revenue.
2. Find the emissions factor with margin corresponding to Tesla's activity
3. Multiply the emissions factor with the revenue

Thought: I will need to find Tesla's main activity and last available revenue first, then search for the emissions factor with the corresponding activity.

Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "Tesla main activity and last available revenue"
}
```


Observation: Mathilde Carlier , Aug 30, 2023. Tesla's revenue grew to nearly 81.5 billion U.S. dollars in the 2022 fiscal year, a 51 percent increase from the previous year. The United States is Tesla's ... Tesla's revenue in the United States, China, and other markets from FY 2018 to FY 2022 (in millions U.S. dollars) Premium Statistic Tesla: vehicle deliveries by main model and quarter 2023 Fiscal year is January - December. Source: Financials a

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0046833...6, -0.0539651571895704]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0046833...6, -0.0539651571895704]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing